In [ ]:
class Bowler:
    def __init__(self, name, average):
        self.name = name
        self.average = float(average)


class Node:
    def __init__(self, cost, h_score, total_cost, runs, total_runs, over, parent_id):
        self.cost = cost
        self.total_cost = total_cost + cost
        self.h_score = h_score
        self.runs = runs
        self.total_runs = total_runs + runs
        self.over = over
        self.a_star_score = self.total_cost + self.h_score
        self.id = str(runs)+"-"+parent_id
        self.parent_id = parent_id

class AStar:
    def __init__(self, target, bowlers, bowling_order):
        self.target = target
        self.bowlers = bowlers
        self.bowling_order = bowling_order
        self.overs = len(bowling_order)

    def _calculate_cost(self, runs, average):
        return ((runs ** 2) / average)

    def _heuristic_score(self, runs_to_score, overs_remaining):
        if overs_remaining == 0:
            return 0
        total_average = 0
        for i in range(self.overs - overs_remaining, self.overs):
            total_average += self.bowlers[self.bowling_order[i]].average

        average_average = total_average / overs_remaining

        if runs_to_score < 0:
            runs_to_score = 0

        return (runs_to_score ** 2) / total_average

    def _create_level(self, selected_node):
        nodes = []
        over = selected_node.over + 1
        total_runs = selected_node.total_runs
        total_cost = selected_node.total_cost
        parent_id = selected_node.id

        for i in range(0, 37):
            cost = self._calculate_cost(
                i, self.bowlers[self.bowling_order[over-1]].average)

            h_score = self._heuristic_score(
                self.target - (total_runs + i), self.overs - over)

            nodes.append(Node(cost, h_score, total_cost, i, total_runs, over, parent_id))

        return nodes

    def find_combination(self):
        open_list = []
        closed_list = []
        root_node = Node(0, self._heuristic_score(self.target, self.overs), 0, 0, 0, 0, "root")

        open_list.append(root_node)

        while (len(open_list) != 0 and open_list[0].total_runs < self.target):
            selected_node = open_list[0]
            #print(selected_node.id)
            if (selected_node.over == self.overs):
                open_list.pop(0)
                continue
            over_outcomes = self._create_level(selected_node)
            open_list.pop(0)
            closed_list.append(selected_node)
            open_list = open_list + over_outcomes
            open_list.sort(key=lambda e: e.a_star_score)

        goal = open_list[0]
        child = goal
        path = [goal]

        while (child.parent_id != "root"):
            for node in closed_list:
                if node.id == child.parent_id:
                    path.append(node)
                    child = node
                    break

        path.reverse()
        path.pop(0)
        return path

In [ ]:
import pandas as pd

In [ ]:
avg = pd.read_csv("avg.csv")

In [ ]:
order = pd.read_csv("order.csv")

In [ ]:
target =[128, 105, 129, 108, 88, 96, 104, 101, 136, 130, 163, 119]
for i in range(1, 13):
  bowlers =[]
  bowling_order =[]
  index = 0
  for bowler in order.loc[(i-1)*10:(i*10)-1, "Order"].unique():
    average = avg.loc[avg["Bowler"] == bowler, "Average"]
    b = Bowler(bowler, average)
    bowlers.append(b)
    bowling_order.append((bowler, index))
    index += 1

  order_to_pass = []
  for bowler in order.loc[(i-1)*10:(i*10)-1, "Order"]:
    for b in bowling_order:
      if b[0] == bowler:
        order_to_pass.append(b[1])

  sample = AStar(target[i-1], bowlers, order_to_pass)
  path = sample.find_combination()
  for node in path:
      print(node.cost, ",", node.runs)

  

5.581180811808118 , 11
6.477577615944806 , 13
5.5175558595531236 , 11
6.220302375809935 , 12
8.922506946588452 , 17
5.581180811808118 , 11
8.922506946588452 , 17
5.5175558595531236 , 11
6.220302375809935 , 12
6.477577615944806 , 13
4.826254826254827 , 10
4.750593824228028 , 10
5.053057099545225 , 10
4.791566842357451 , 10
5.053057099545225 , 10
6.36055701919458 , 13
4.826254826254827 , 10
4.750593824228028 , 10
4.791566842357451 , 10
5.818181818181818 , 12
4.3365134431916745 , 10
5.135823429541596 , 11
5.4961832061068705 , 12
4.703668861712135 , 10
4.3365134431916745 , 10
3.8099717779868296 , 9
9.877551020408163 , 22
5.4961832061068705 , 12
5.135823429541596 , 11
9.877551020408163 , 22
4.3308791684712 , 10
5.378739656269891 , 13
4.704510108864697 , 11
5.3019145802650955 , 12
3.245436105476674 , 8
4.3308791684712 , 10
4.704510108864697 , 11
5.3019145802650955 , 12
5.378739656269891 , 13
3.245436105476674 , 8
3.5494279847462598 , 11
2.179715302491103 , 7
2.192393736017897 , 7
3.555686159

In [ ]:
for i in range(1,13):
  print((i-1)*10, (i*10)-1)

0 9
10 19
20 29
30 39
40 49
50 59
60 69
70 79
80 89
90 99
100 109
110 119


In [ ]:
bowling_order

[('Mahesh Theekshana', 0),
 ('Wayne Parnell', 1),
 ('Wahab Riaz', 2),
 ('Rayad Emrit', 3),
 ('Fabian Allen', 4)]

In [ ]:
AStar(60, bowlers, [0, 1, 2, 3])